## Imports

In [86]:
import pandas as pd
import numpy as np
import tabula
import configparser
import os


In [87]:
cwd = os.getcwd()

## Config

In [88]:
#Creating configuration
config_path = os.path.join(cwd, '../docs/config.ini')
config = configparser.ConfigParser()
config.read(config_path)

['c:\\Users\\Tony\\OneDrive\\Python\\Projects\\financialstatements\\financialstatements\\notebooks\\../docs/config.ini']

## Utility Functions

In [89]:
def creating_input_paths(directory):
    files = os.listdir(directory)
    paths = []
    for file in files:
        path = os.path.join(directory, file)
        paths.append(path)
    return paths

In [90]:
def debit_credit_check(df):
    debit_total = df['DEBIT'].sum()
    credit_total = df['CREDIT'].sum()
    return debit_total == credit_total

In [91]:
def creating_output(df, month_df, type, config, cwd):
    df['Month_Num'] = df['Date'].dt.month
    df['Transaction_ID'] = type + '-' + \
        df['Month_Num'].astype("str") + '-' + (df.index + 1).astype("str")
    df = pd.merge(df,
                  month_df,
                  on='Month_Num',
                  how='left')
    months = df['Month'].unique()
    file_path = os.path.join(cwd, config.get(
        "data_outputs_directory", "JOURNAL_ENTRIES"))
    month_num = 1
    for month in months:
        file_df = df[df['Month'] == month]
        file_df = file_df[['Transaction_ID', 'Date', 'GL_Code',
                           'Account', 'Description', 'DEBIT', 'CREDIT']]
        file_name = f'{month_num}_{type}_{month}_entries.csv'
        file_df.to_csv(os.path.join(file_path, file_name), index=False)
        month_num += 1

## Paystub Functions

In [92]:
def creating_df(paths):
    dfs = []
    for path in paths:
        tables = tabula.read_pdf(path, pages = 'all', area = [396, 36, 756, 612], guess = False)
        df = pd.concat(tables).reset_index(drop=True)
        df['Date'] = path[-14:-4]
        dfs.append(df)
    return pd.concat(dfs).reset_index(drop=True)

In [93]:
def find_deductions(df):
    df = df[['Unnamed: 4', 'Unnamed: 5', 'Date']]
    df = df.rename(columns = {'Unnamed: 4': 'Item', 'Unnamed: 5':'Amount'})
    df = df[1:]
    df = df.dropna().reset_index(drop = True)
    df = df.query("Item != 'Total' and Item != 'DEDUCTIONS' and Item != 'CURRENT'")
    return df

In [94]:
def find_earnings(df):
    df = df[['Unnamed: 0', 'Unnamed: 3', 'Date']]
    df = df.rename(columns = {'Unnamed: 0': 'Item', 'Unnamed: 3':'Amount'})
    df = df[1:]
    df = df.dropna().reset_index(drop = True)
    df = df.query("Item != 'Total' and Item != 'TAX' and Item != 'EARNINGS'")
    return df

In [95]:
def processing_df(deductions_df, earnings_df, coa_purch_df):
    df = pd.concat([deductions_df, earnings_df]).reset_index(drop=True)
    df = pd.merge(df, 
              coa_purch_df,
                on = 'Item',
                how = 'left')
    df = df.rename(columns = {'Item': 'Description'})
    df = df[['Date', 'GL_Code', 'Account', 'Description', 'Amount', 'Category', 'Account_Type', 'Order_Col']]
    df['Date'] = pd.to_datetime(df['Date'])
    df['Amount'] = df['Amount'].str.replace('$', '', regex=False)
    df['DEBIT'] = np.where(df['Account_Type'].isin(['Asset', 'Deduction']), df['Amount'], np.nan)
    df['CREDIT'] = np.where(df['Account_Type'] == 'Revenue', df['Amount'],np.nan)
    df['DEBIT'] = df['DEBIT'].str.replace(',', '').astype('float')
    df['CREDIT'] = df['CREDIT'].str.replace(',', '').astype('float')
    return df

In [96]:
def creating_cash_entries(df):
    df_grouped = df.groupby('Date', as_index=False)[['DEBIT', 'CREDIT']].sum()
    df_grouped['DEBIT'] = df_grouped['CREDIT'] - df_grouped['DEBIT']
    df_grouped['CREDIT'] = np.nan
    df_grouped['GL_Code'] = 100101
    df_grouped['Account'] = 'Free Checking Bank OZK'
    df_grouped['Description'] = 'Cash from paystub'
    df_grouped['Amount'] = df_grouped['DEBIT']
    df_grouped['Category'] = 'Cash'
    df_grouped['Account_Type'] = 'Asset'
    df_grouped['Order_Col'] = 1
    df_grouped = df_grouped[['Date', 'GL_Code', 'Account', 'Description',
       'Amount', 'Category', 'Account_Type', 'Order_Col','DEBIT', 'CREDIT']]
    return pd.concat([df, df_grouped]).reset_index(drop=True)

## Reading Data

### Paystubs

In [97]:
PAYSTUB_DIRECTORY = os.path.join(cwd, config.get("data_inputs_directory", "PAYSTUB_DIRECTORY"))

In [98]:
paths = creating_input_paths(PAYSTUB_DIRECTORY)

### Table Data

In [99]:
COA_DATA = config.get('table_files', 'COA_DATA')
MONTH_DATA = config.get('table_files', 'MONTH_DATA')

In [100]:
coa_purch_df = pd.read_excel(COA_DATA, sheet_name='coa_paystub_link_table')

In [101]:
month_df = pd.read_excel(MONTH_DATA)

## ETL

In [102]:
df = creating_df(paths)

In [103]:
deductions_df = find_deductions(df)

In [104]:
earnings_df = find_earnings(df)

In [105]:
df = processing_df(deductions_df, earnings_df, coa_purch_df)

C:\Users\Tony\AppData\Local\Temp\ipykernel_16036\1881938269.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Amount'] = df['Amount'].str.replace('$', '')


In [106]:
df = creating_cash_entries(df)

In [107]:
df = df.sort_values(by=['Date', 'Order_Col']).reset_index(drop=True)

In [109]:
type = 'paystub'

In [110]:
creating_output(df, month_df, type, config, cwd)